<a href="https://colab.research.google.com/github/dummy26/face-mask-detector/blob/master/FaceMask1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Upload kaggle json file downloaded from kaggle here
from google.colab import files
files.upload()
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d andrewmvd/face-mask-detection
! unzip face-mask-detection.zip

In [ ]:
!pip install tf-nightly
import matplotlib.pyplot as plt
import numpy as np
import os
import shutil
from bs4 import BeautifulSoup
import random
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
images_path = 'images'
labels_path = 'annotations'

In [ ]:
#Considering mask worn incorrectly as with_mask
# 0- mask
# 1 - no mask
def generate_label(obj):
    if obj.find('name').text == "with_mask" or obj.find('name').text == 'mask_weared_incorrect':
        return 0
    return 1

In [ ]:
labels = []
for label_path in sorted(os.listdir(labels_path)):
    xml_file = os.path.join(labels_path, label_path)
    with open(xml_file) as f:
        data = f.read()
        soup = BeautifulSoup(data, 'xml')
        obj = soup.find('object')
        labels.append(generate_label(obj))

In [ ]:
unique_labels, counts = np.unique(labels, return_counts=True)
print(unique_labels, counts)

In [ ]:
images = [plt.imread(os.path.join(images_path, img_path)) for img_path in sorted(os.listdir(images_path))]
nrows, ncols = 5, 5
figsize = [30, 30]    

fig, ax = plt.subplots(nrows=nrows, ncols=ncols, figsize=figsize)
for i, axi in enumerate(ax.flat):
    random_idx = random.randint(0, len(images)-1)
    img = images[random_idx]
    axi.set_title(labels[random_idx])
    axi.imshow(img)
    axi.axis('off')

plt.show()

In [ ]:
# w = []
# h = []
# for img in images:
#     w.append(img.shape[0])
#     h.append(img.shape[1])

# print(min(w), min(h), max(w), max(h))

In [ ]:
!mkdir FaceMaskDataset
%cd FaceMaskDataset
!mkdir Mask
!mkdir NoMask
%cd /content

In [ ]:
mask_path = 'FaceMaskDataset/Mask'
no_mask_path = 'FaceMaskDataset/NoMask'

In [ ]:
IMG_PATHS = sorted(os.listdir(images_path))
for i in range(len(labels)):
    img = os.path.join(images_path, IMG_PATHS[i])
    if labels[i] == 0:
        shutil.copy(img, mask_path)
    elif labels[i] == 1:
        shutil.copy(img, no_mask_path)

In [ ]:
# masks = sorted(os.listdir(mask_path))
# nrows, ncols = 5, 5
# figsize = [30, 30]    

# fig, ax = plt.subplots(nrows=nrows, ncols=ncols, figsize=figsize)
# for i, axi in enumerate(ax.flat):
#     random_idx = random.randint(0, len(masks)-1)
#     img = plt.imread(os.path.join(mask_path, masks[random_idx]))
#     axi.set_title(labels[random_idx])
#     axi.imshow(img)
#     axi.axis('off')

# plt.show()

In [ ]:
# 0 is mask and 1 is no mask 
image_size = (180, 180)
batch_size = 32

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "FaceMaskDataset",
    validation_split=0.1,
    subset="training",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
    label_mode='binary',
)
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "FaceMaskDataset",
    validation_split=0.1,
    subset="validation",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
    label_mode='binary',
)

train_ds = train_ds.prefetch(buffer_size=32)
val_ds = val_ds.prefetch(buffer_size=32)

In [ ]:
# plt.figure(figsize=(10, 10))
# for images, labels in train_ds.take(1):
#     for i in range(9):
#         ax = plt.subplot(3, 3, i + 1)
#         plt.imshow(images[i].numpy().astype("uint8"))
#         plt.title(int(labels[i]))
#         plt.axis("off")

In [ ]:
data_augmentation = keras.Sequential(
    [
        layers.experimental.preprocessing.RandomFlip("horizontal"),
        layers.experimental.preprocessing.RandomRotation(0.1),
    ]
)

# plt.figure(figsize=(10, 10))
# for images, _ in train_ds.take(1):
#     augmented_images = data_augmentation(images)
#     for i in range(9):
#         ax = plt.subplot(3, 3, i + 1)
#         plt.imshow(augmented_images[0].numpy().astype("uint8"))
#         plt.axis("off")

In [ ]:
def make_model(input_shape, num_classes):
    inputs = keras.Input(shape=input_shape)
    # Image augmentation block
    x = data_augmentation(inputs)

    # Entry block
    x = layers.experimental.preprocessing.Rescaling(1.0 / 255)(x)
    x = layers.Conv2D(32, 3, strides=2, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    x = layers.Conv2D(64, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    previous_block_activation = x  # Set aside residual

    for size in [128, 256, 512, 728]:
        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(size, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(size, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.MaxPooling2D(3, strides=2, padding="same")(x)

        # Project residual
        residual = layers.Conv2D(size, 1, strides=2, padding="same")(
            previous_block_activation
        )
        x = layers.add([x, residual])  # Add back residual
        previous_block_activation = x  # Set aside next residual

    x = layers.SeparableConv2D(1024, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    x = layers.GlobalAveragePooling2D()(x)
    if num_classes == 2:
        activation = "sigmoid"
        units = 1
    else:
        activation = "softmax"
        units = num_classes

    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(units, activation=activation)(x)
    return keras.Model(inputs, outputs)


model = make_model(input_shape=image_size + (3,), num_classes=2)
# keras.utils.plot_model(model, show_shapes=True)

In [ ]:
epochs = 10

# callbacks = [
#     keras.callbacks.ModelCheckpoint("save_at_{epoch}.h5"),
# ]
model.compile(
    optimizer=keras.optimizers.Adam(1e-3),
    loss="binary_crossentropy",
    metrics=["accuracy"],
)
model.fit(train_ds, epochs=epochs, validation_data=val_ds)

In [ ]:
path = "images/maksssksksss812.png"
img = keras.preprocessing.image.load_img(path, target_size=image_size)
img_array = keras.preprocessing.image.img_to_array(img)
img_array = tf.expand_dims(img_array, 0)  # Create batch axis

predictions = model.predict(img_array)
score = predictions[0][0]
percent_mask =  100 * (1-score)  #cuz 0 means mask
print(f'{percent_mask:.3f}% mask')
img = plt.imread(path)
plt.imshow(img)
plt.show()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
drive_path = '/content/drive/My Drive'
%cd /content/drive/My\ Drive
!mkdir FaceMaskDataset
%cd FaceMaskDataset
!mkdir Mask
!mkdir NoMask

drive_mask_path = os.path.join(drive_path, mask_path)
for f in os.listdir(mask_path):
    shutil.copy(os.path.join(mask_path, f), drive_mask_path)


drive_no_mask_path = os.path.join(drive_path, no_mask_path)
for f in os.listdir(no_mask_path):
    print(f)
    shutil.copy(os.path.join(no_mask_path, f), drive_no_mask_path)


%cd /content